In [1]:
import pandas as pd
import numpy as np
import sklearn
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

tf.random.set_seed(1)

In [2]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
import re
nltk.download('stopwords')
stops = set(stopwords.words('english'))
stops.add("'s")
stops.remove("not")

def clean(sentence):
    # delete stopwords
    temp = " ".join(filter(lambda x: x not in stops, sentence.split()))
    # Remove punctuation
    temp = temp.translate(str.maketrans('', '', punctuation))
    # remove non-english characters
    temp = temp.encode("ascii", "ignore").decode()
    # Change all to lower case
    temp = temp.lower()
    # Delete numbers
    temp = re.sub(r'[0-9]', "", temp)
    # Delete excessive spaces and return
    return re.sub("  ", " ", temp)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wmy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
articles = pd.read_csv("Article-2022-05-08.csv")
articles_labeled = articles.copy()
articles_labeled['key_terms'] = articles['key_terms'].apply(lambda s: [l for l in str(s).split(',')])
mlb = MultiLabelBinarizer()
mlb.fit(articles_labeled["key_terms"])
y_bin = mlb.transform(articles_labeled["key_terms"])
articles_labeled['text'] = articles_labeled['text'].fillna("")
articles_labeled['text'] = articles_labeled['title'] + articles_labeled['text']
articles_labeled["text"] = articles_labeled["text"].apply(clean)


In [6]:
data = tf.data.Dataset.from_tensor_slices((articles_labeled["text"], y_bin))
data = data.shuffle(buffer_size = len(data), seed=1)

train_size = int(0.7*len(data))
val_size   = int(0.1*len(data))

train = data.take(train_size)
val   = data.skip(train_size).take(val_size)
test  = data.skip(train_size + val_size)

In [7]:
# only the top distinct words will be tracked
max_tokens = 2000

# each headline will be a vector of length 25
sequence_length = 25

vectorize_layer = TextVectorization(
    max_tokens=max_tokens, # only consider this many words
    output_mode='int',
    output_sequence_length=sequence_length) 

headlines = train.map(lambda x, y: x)
vectorize_layer.adapt(headlines)

In [8]:
new_model = tf.keras.models.load_model('saved_model/classfication_model_multi_label.h5')

In [9]:
keys = dict(zip(range(16), [int(i) for i in ['1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4', '5',
       '6', '7', '8', '9']]))

def idx_to_label(arr):
    return [[keys[i] for i in range(16) if j[i] == 1] for j in arr]

def predict(x):
    """
    predict labels
    @params x: 1d or 2d array or list
    @return labels in a 2d array
    """
    x = np.array(x).reshape(-1,)
    x = np.array(list(map(clean, x)))
    x = tf.expand_dims(x, -1)
    return idx_to_label(np.round(new_model.predict(vectorize_layer(x))))  

In [10]:
predict(["""
FedEx Commits to Carbon-Neutral Operations by the year 2040

FedEx Corp. announced today an ambitious goal to achieve carbon-neutral operations globally by 2040.

To help reach this goal, FedEx is designating more than $2 billion of initial investment in three key areas: vehicle electrification, sustainable energy, and carbon sequestration. This includes a pledge of $100 million to Yale University to help establish the Yale Center for Natural Carbon Capture, accelerating research into methods of carbon sequestration at scale, with an initial focus on helping to offset greenhouse gas emissions equivalent to current airline emissions.

?€?We have a responsibility to take bold action in addressing climate challenges,?€? said Frederick W. Smith, Chairman and CEO, FedEx Corp. ?€?This goal builds on our longstanding commitment to sustainability throughout our operations, while at the same time investing in long-term, transformational solutions for FedEx and our entire industry.?€?

Key steps toward reaching the carbon neutral goal include:

Vehicle Electrification . By 2040, the entire FedEx parcel pickup and delivery (PUD) fleet will be zero-emission electric vehicles. This will be accomplished through phased programs to replace existing vehicles. For example, by 2025, 50% of FedEx Express global PUD vehicle purchases will be electric, rising to 100% of all purchases by 2030.

Sustainable Customer Solutions . FedEx will work with customers to offer end-to-end sustainability for their supply chains through carbon-neutral shipping offerings and sustainable packaging solutions.

Sustainable Fuels . FedEx will continue to invest in alternative fuels to reduce aircraft and vehicle emissions.

Fuel Conservation and Aircraft Modernization . FedEx will build on its successful FedEx Fuel Sense initiatives designed to reduce fuel consumption in its aircraft. Since 2012, the FedEx Fuel Sense and Aircraft Modernization programs have saved a combined 1.43 billion gallons of jet fuel and avoided over 13.5 million metric tons of carbon dioxide (CO2) emissions.

Facilities . FedEx will continue efforts to make its more than 5,000 facilities worldwide more sustainable through continued investments in efficient facilities, renewable energy, and other energy management programmes.

Natural Carbon Sequestration . FedEx funding will help to establish the Yale Center for Natural Carbon Capture to support applied research into natural carbon sequestration solutions.

The path toward sustainability requires new strategies for removing and storing Earth?€?s excess carbon. The Yale Center for Natural Carbon Capture will catalyze interdisciplinary research across the natural sciences and engineering in an effort to accelerate this work.

Center researchers will develop methods that build on natural carbon storage systems, including biological ecosystems and the geological carbon cycle, improving, where possible, how quickly carbon can be absorbed, how much can be contained, and how long it can be stored. Through these efforts, Yale scientists aim to create a portfolio of carbon removal strategies that have impacts on a global scale.

Building upon initial successes in the aviation sector, the centre will broaden its scope to address additional global sources of emissions ?€? publishing and sharing its findings so that businesses, industries, and governments can benefit from work that will accelerate the adoption and implementation of natural carbon capture strategies around the world.

?€?Addressing climate change is a complex challenge that demands urgent action, and natural carbon capture strategies will be one key part of that action,?€? said Dr. Ingrid C. ?€?Indy?€? Burke, the Carl W. Knobloch, Jr. Dean of the Yale School of the Environment. ?€?Through the creation of the Yale Center for Natural Carbon Capture, we aim to develop measurable carbon capture strategies to help offset carbon emissions globally.?€?

The FedEx commitment builds on a history of sustainable practices. Since 2009, the company?€?s efforts have contributed to an approximately 40% reduction in CO2 emissions intensity across the enterprise while package volume increased 99% during that period. Recently, FedEx was ranked first in its industry on JUST Capital?€?s 2021 list of ?€?America?€?s Most Just Companies?€? in the environment category and first in the travel, transport and logistics sector of Newsweek?€?s ?€?America?€?s Most Responsible Companies 2021.?€?

?€?While we?€?ve made great strides in reducing our environmental impact, we have to do more. The long-term health of our industry is directly linked to the health of the planet, but this effort is about more than the bottom line ?€? it?€?s the right thing to do,?€? said Mitch Jackson, Chief Sustainability Officer, FedEx Corp. ?€?At FedEx, we are committed to connecting people and possibilities resourcefully and responsibly. The steps we are taking today will contribute a positive impact for generations to come.?€?

Disclaimer: This media release is auto-generated. The CSR Journal is not responsible for the content
"""])

1/1 [==============================] - 5s 5s/step


[[5]]